In [117]:
pip install emoji


In [118]:
pip install nltk


In [119]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import emoji
import os
from os import path
from google.colab import files
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [121]:
doc = pd.read_csv("/content/drive/MyDrive/dataset/300DataRev.csv")
doc.head(5)

,docID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,1,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,2,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,3,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,4,B00005ML71,"Wilhelmina Zeitgeist ""coolartsybabe""","[0, 0]",I bought this to use with my keyboard. I wasn'...,2,Definitely Not For The Seasoned Piano Player,1376697600,"08 17, 2013"
4,5,B000068NSX,"C.E. ""Frank""","[0, 0]",This Fender cable is the perfect length for me...,4,Durable Instrument Cable,1376352000,"08 13, 2013"


In [122]:
#Penambahan Label D (Dokumen)
doc = doc.loc[:, ['docID', 'reviewText']]
doc["docID"] = ["D" + str(ind) for ind in range(1, len(doc) + 1)]
doc.head(5)

,docID,reviewText
0,D1,"Not much to write about here, but it does exac..."
1,D2,The product does exactly as it should and is q...
2,D3,The primary job of this device is to block the...
3,D4,I bought this to use with my keyboard. I wasn'...
4,D5,This Fender cable is the perfect length for me...


In [123]:
#penghapusan emoji
def remove_emoji_and_emoticons(string):
    string = emoji.replace_emoji(string, replace='')
    emoticon_pattern = re.compile(r'[:;=][oO\-]?[\)\(\[\]DdpP]')
    string = emoticon_pattern.sub(r'', string)
    return string

doc['reviewText'] = doc['reviewText'].apply(remove_emoji_and_emoticons)
doc['reviewText'] = doc['reviewText'].str.strip()

doc

,docID,reviewText
0,D1,"Not much to write about here, but it does exac..."
1,D2,The product does exactly as it should and is q...
2,D3,The primary job of this device is to block the...
3,D4,I bought this to use with my keyboard. I wasn'...
4,D5,This Fender cable is the perfect length for me...
...,...,...
295,D296,"This is your basic, black electric guitar cabl..."
296,D297,"Hosa cable quality can be all over the place, ..."
297,D298,I have many lengths of these Hosa cables and h...
298,D299,I've been using these for about 3 weeks now - ...


In [124]:
# menghapus koma, angka, non Alpha-numeric, karakter spesial, dan mengubah ke lower case
doc['reviewText'] = doc['reviewText'].str.replace(",", " ")
doc['reviewText'] = doc['reviewText'].str.replace(r'\d+', '', regex=True)
doc['reviewText'] = doc['reviewText'].str.replace(r'[^\w\s]', '', regex=True)
doc['reviewText'] = doc['reviewText'].str.replace(r'[\$#&\.]', '', regex=True)
doc['reviewText'] = doc['reviewText'].str.strip().str.lower()

doc = doc.loc[:, ['docID', 'reviewText']]
doc

,docID,reviewText
0,D1,not much to write about here but it does exac...
1,D2,the product does exactly as it should and is q...
2,D3,the primary job of this device is to block the...
3,D4,i bought this to use with my keyboard i wasnt ...
4,D5,this fender cable is the perfect length for me...
...,...,...
295,D296,this is your basic black electric guitar cabl...
296,D297,hosa cable quality can be all over the place ...
297,D298,i have many lengths of these hosa cables and h...
298,D299,ive been using these for about weeks now the...


In [125]:
#Tokenisasi
doc['reviewText'] = doc['reviewText'].apply(lambda x: word_tokenize(str(x)))

doc

,docID,reviewText
0,D1,"[not, much, to, write, about, here, but, it, d..."
1,D2,"[the, product, does, exactly, as, it, should, ..."
2,D3,"[the, primary, job, of, this, device, is, to, ..."
3,D4,"[i, bought, this, to, use, with, my, keyboard,..."
4,D5,"[this, fender, cable, is, the, perfect, length..."
...,...,...
295,D296,"[this, is, your, basic, black, electric, guita..."
296,D297,"[hosa, cable, quality, can, be, all, over, the..."
297,D298,"[i, have, many, lengths, of, these, hosa, cabl..."
298,D299,"[ive, been, using, these, for, about, weeks, n..."


In [126]:
#Penghilangan Stopword
stop_words = set(stopwords.words('english'))
doc['reviewText'] = doc['reviewText'].apply(lambda x: [word for word in x if word not in stop_words])

doc

,docID,reviewText
0,D1,"[much, write, exactly, supposed, filters, pop,..."
1,D2,"[product, exactly, quite, affordablei, realize..."
2,D3,"[primary, job, device, block, breath, would, o..."
3,D4,"[bought, use, keyboard, wasnt, really, aware, ..."
4,D5,"[fender, cable, perfect, length, sometimes, fi..."
...,...,...
295,D296,"[basic, black, electric, guitar, cable, works,..."
296,D297,"[hosa, cable, quality, place, products, good, ..."
297,D298,"[many, lengths, hosa, cables, neverhad, proble..."
298,D299,"[ive, using, weeks, strong, flexible, dont, se..."


In [127]:
#Stemming memakai Snowball Stemmer
stemmer = SnowballStemmer("english")
doc['reviewText'] = doc['reviewText'].apply(lambda x: [stemmer.stem(word) for word in x])

doc

,docID,reviewText
0,D1,"[much, write, exact, suppos, filter, pop, soun..."
1,D2,"[product, exact, quit, affordablei, realiz, do..."
2,D3,"[primari, job, devic, block, breath, would, ot..."
3,D4,"[bought, use, keyboard, wasnt, realli, awar, o..."
4,D5,"[fender, cabl, perfect, length, sometim, find,..."
...,...,...
295,D296,"[basic, black, electr, guitar, cabl, work, gre..."
296,D297,"[hosa, cabl, qualiti, place, product, good, so..."
297,D298,"[mani, length, hosa, cabl, neverhad, problem, ..."
298,D299,"[ive, use, week, strong, flexibl, dont, seem, ..."


In [128]:
# Pengganti Lemmatization
custom_words = {
    "rapid", "clearer", "sunlight", "effort", "avoid", "realtimesoft", "cleaner", "out", "tempt", "junk",
    "run", "welltravel", "champion", "task", "later", "origin", "standon", "outdoor", "what", "soon", "sell",
    "effect", "expand", "scarlett", "distorion", "ocean", "backlight", "ask", "plus", "pickup", "slow", "view",
    "ok", "he", "footswitch", "wear", "made", "moment", "space", "hardwar", "characterist", "unwant", "will",
    "bolder", "notic", "shouldnt", "page", "catch", "fear", "head", "yesterday", "bandfirst", "arm", "specif",
    "overgrab", "satriani", "size", "thumbwheel", "project", "electroharmonix", "cram", "jack", "wow", "mute",
    "bitch", "easiergreat", "popularslight", "swivel", "disappoint", "timekmc", "mask", "sps", "opt", "felt",
    "joe", "dude", "regret", "strip", "kink", "storag", "jacket", "conceal", "principl", "budget", "compliant",
    "usbba", "own", "stratther", "respond", "fluctuat", "orang", "ministrat", "fenderp", "gibson", "imposs",
    "guitarlel", "hotel", "bridg", "pancak", "thereprob", "wireless", "rattl", "surprisemi", "reenabl", "newid",
    "technolog", "instead", "gradual", "factori", "ahead", "clawhamm", "ie", "rangebut", "gripe", "habit", "import",
    "tour", "unplug", "drawer", "goo", "avail", "frank", "stash", "finer", "fxs", "spider", "japan", "crack",
    "connector", "surpri", "boo", "longtim", "given", "extran", "lamin", "ga", "would", "xp", "might", "gsb",
    "vcwebber", "web", "allanalog", "short", "greenback", "aluminum", "anchor", "technica", "rigor", "align", "rid",
    "bottl", "notat", "ident", "useless", "guitar", "small", "mar", "pianoand", "rest", "environ", "gone", "uniqu",
    "reusabl", "control", "suffici", "advi", "retro", "mold", "pole", "drastic", "process", "postover", "either",
    "seek", "nanoweb", "feedback", "vocal", "child", "yellow", "code", "comment", "seem", "predict", "lite", "preset",
    "deepbowl", "depress", "compress", "smell", "baseman", "guitarist", "brilliant", "object", "nonstudio", "micron",
    "skeptic", "plugin", "werent", "world", "champ", "late", "idea", "colour", "butterfield", "chain", "seven", "rule",
    "towel", "surpass", "mexican", "nicer", "versus", "play", "shot", "follow", "use", "betterwear", "restart",
    "written", "walmart", "leader", "shown", "obvious", "wellknown", "intent", "rehear", "sit", "weight", "attack",
    "beater", "screamer", "display", "england", "rare", "treat", "uppermid", "scoop", "midq", "gain", "write", "flesh",
    "love", "printer", "min", "game", "road", "dust", "trash", "progress", "construct", "count", "transport", "youth",
    "neckbottom", "complain", "signal", "hardpress", "back", "function", "wallet", "tap", "fall", "thank", "knowth",
    "familiar", "fullsound", "neverhad", "lock", "explor", "consist", "least", "defect", "cheaper", "hd", "regardless",
    "pain", "cart", "spray", "wall", "bos", "toy", "neither", "cave", "model", "match", "bump", "strike", "weird",
    "tast", "afternoon", "grate", "tradeoff", "near", "app", "generat", "worldjust", "glad", "word", "version", "ball",
    "mini", "market", "repeat", "tax", "monkey", "maximum", "spin", "mother", "two", "somehow", "cutaway", "loud",
    "slide", "place", "various", "fx", "stay", "greatest", "smaller", "debat", "gorgeous", "seller", "claim", "begin",
    "darn", "bag", "stylist", "stair", "headset", "kit", "yearold", "ding", "solder", "bigger", "user", "enthusiast",
    "ten", "plane", "starsrock", "best", "martin", "dm", "adjust", "charm", "deal", "manifest", "absolut", "array",
    "trade", "bestest", "motion", "team", "china", "god", "disconnect", "offset", "punk", "conclud", "english",
    "insult", "fair", "horror", "ribbon", "rod", "well", "breakup", "tripod", "neck", "prematur", "comfort", "hole",
    "press", "buy", "driver", "brand", "interest", "via", "bind", "aggress", "note", "station", "sustain", "junior",
    "yes", "type", "enough", "fender", "win", "understand", "medium", "plastic", "stuck", "tank", "even", "among",
    "find", "boot", "drink", "manner", "nest", "bare", "intrument", "ada", "aspect", "butter", "term", "money", "want",
    "fulli", "belong", "badass", "jazz", "bow", "silent", "convert", "menu", "adaptor", "bus", "better", "stick",
    "strong", "clean", "tube", "british", "away", "broadcast", "outstand", "instruct", "detect", "week", "pop",
    "phaser", "workover", "self", "unwrap", "paper", "flash", "longer", "special", "mixer", "metal", "buff", "care",
    "night", "neon", "obtain", "lowest", "there", "am", "american", "pawn", "read", "port", "priceless", "slightest",
    "bodyneck", "wait", "yet", "slider", "solidwood", "human", "laptop", "brillianc", "curv", "fabric", "sleep",
    "edit", "mate", "spectacular", "similar", "ram", "insert", "actual", "phone", "strang", "hollow", "sound", "paid",
    "print", "crew", "forget", "relief", "somewhat", "time", "abut", "rocket", "wire", "box", "ish", "level", "room",
    "scout", "abandon", "cheap", "forth", "twice", "thousand", "program", "line", "sidemount", "girl", "softer",
    "right", "keep", "bug", "direct", "male", "sister", "map", "rat", "water", "crystal", "stack", "heck", "price",
    "uncoat", "way", "cc", "embarrass", "separ", "rout", "length", "visual", "system", "holder", "wah", "mount",
    "touchrespon", "tell", "chorus", "pod", "fuzz", "noic", "knob", "modul", "rock", "show", "access", "done", "bypass",
    "screen", "pedal", "blow", "most", "one", "modern", "track", "tape", "comb", "your", "gadget", "walk", "three",
    "introduct", "tunestil", "jumpi", "plan", "strummer", "accid", "soft", "typic", "bash", "hint", "exact", "coil",
    "safest", "oil", "close", "friday", "busk", "buffer", "reader", "age", "attach", "major", "problem", "blown",
    "wash", "hold", "eject", "return", "travel", "remark", "addon", "bean", "float", "tailor", "throw", "knock",
    "think", "string", "email", "beyond", "keeper", "addict", "stress", "graphic", "next", "center", "hot", "nick",
    "nuclear", "tech", "enjoy", "music", "macbook", "split", "winner", "board", "bedroom", "unzip", "addit", "grab",
    "pure", "custom", "low", "pass", "first", "except", "door", "life", "instant", "speed", "favor", "alloy", "fell",
    "temperatur", "spring", "side", "rain", "sick", "needless", "son", "base", "upper", "speaker", "movement",
    "numer", "behind", "blade", "due", "worth", "top", "happier", "weak", "pyramid", "opinion", "photo", "state",
    "technic", "disk", "tip", "test", "check", "brother", "grape", "store", "shop", "save", "hard", "node", "pay",
    "bit", "serious", "thee", "paperwork", "east", "cup", "setup", "add", "stark", "forum", "cricket", "skype",
    "audio", "eight", "acid", "end", "fine", "taken", "turn", "round", "acoust", "contrast", "knockoff", "skill",
    "jump", "drive", "tame", "question", "front", "louder", "comfi", "error", "stage", "have", "feat", "ago", "strict",
    "clone", "perform", "cake", "silver", "search", "club", "seen", "seldom", "cour", "inch", "larger", "finish",
    "flat", "hand", "none", "mic", "went", "research", "ground", "shimmer", "cool", "uncommon", "heil", "fuel", "book",
    "red", "lay", "case", "analog", "warm", "outer", "trigger", "flaw", "white", "cap", "appear", "punch", "mix",
    "speak", "turner", "lab", "sea", "luck", "alien", "ruler", "gleam", "algorithm", "inc", "accept", "dead",
    "relationship", "tear", "common", "train", "scale", "dist", "fake", "yard", "broad", "mint", "spend", "less",
    "deep", "poster", "respon", "trust", "record", "repair", "blast", "workaround", "far", "dollar", "stuff",
    "thicker", "indonesia", "homestudio", "poke", "combo", "report", "radio", "fight", "unbox", "grandson", "kill",
    "local", "legal", "total", "background", "bolt", "across", "mid", "overload", "hello", "brick", "topic", "give",
    "date", "curious", "popular", "full", "pick", "yeah", "skin", "fact", "spot", "action", "last", "rich", "tea",
    "closet", "portion", "recent", "stomp", "open", "older", "theori", "option", "crash", "mod", "suspect", "assembl",
    "tune", "panel", "thin", "drag", "wife", "ruin", "tock", "excess", "dresser", "brighter", "lennon", "standard",
    "chosen", "passion", "uniform", "said", "tweak", "mim", "profil", "much", "drill", "label", "flex", "button",
    "zip", "outlast", "dell", "output", "reproduct", "member", "broke", "thought", "risk", "amount", "half", "excel",
    "summer", "thick", "reconnect", "guesswork", "punish", "live", "reset", "four", "found", "window", "extra",
    "wound", "counter", "sad", "nut", "patent", "earth", "rush", "ever", "zoom", "hear", "product", "lantern", "job",
    "break", "circuit", "dual", "lube", "basic", "afford", "trick", "bulk", "sweet", "desktop", "over", "extend",
    "bad", "coat", "key", "tinker", "desk", "cross", "suit", "paul", "lost", "grow", "also", "wild", "look", "factor",
    "stink", "wrote", "benefit", "wind", "jam", "develop", "crap", "fold", "zero", "bravo", "again", "friend",
    "expect", "period", "dose", "underneath", "song", "curtain", "asleep", "year", "rig", "ratio", "switch", "gear",
    "real", "number", "young", "socket", "broken", "respect", "pro", "patch", "adult", "correct", "ton", "sever",
    "bright", "phantom", "heaven", "drumset", "bet", "load", "hey", "sort", "voic", "hybrid", "awar", "may", "volt",
    "highgain", "class", "draw", "shift", "student", "free", "hidden", "start", "quit", "beet", "cat", "golden",
    "wide", "mark", "sold", "everyday", "filter", "elixir", "perfect", "overshift", "dave", "cold", "steel", "creat",
    "driven", "blue", "talk", "score", "rather", "stop", "normal", "twin", "know", "aqua", "call", "unit", "seal",
    "folk", "channel", "processor", "musician", "harder", "who", "mention", "star", "truth", "saw", "dive", "fun",
    "make", "meet", "art", "strength", "makeshift", "unclip", "eye", "hate", "spare", "download", "fraction", "six",
    "smash", "second", "backup", "monster", "monitor", "part", "success", "block", "mine", "rate"
}

def keep_custom_words(text, custom_words):
    custom_words_set = set(word.lower() for word in custom_words)
    words = re.findall(r'\b\w+\b', text.lower())
    kept_words = [word for word in words if word in custom_words_set]

    return ' '.join(kept_words)

doc['reviewText'] = doc['reviewText'].apply(lambda x: keep_custom_words(str(x), custom_words))

doc


,docID,reviewText
0,D1,much write exact filter pop sound record much ...
1,D2,product exact quit screen even better one scre...
2,D3,job block would pop sound voic pass notic filt...
3,D4,use awar option pedal pedal acoust use either ...
4,D5,fender perfect length find bit know last gripe...
...,...,...
295,D296,basic guitar far end sound enough problem late...
296,D297,place product sound last product sound break p...
297,D298,length neverhad problem musician stage comment...
298,D299,use week strong seem develop kink habit seen u...


In [129]:
# Inisialisasi Lokasi dokumen Tiap kata
inverted_index = {}

# Iterasi melalui dataframe
for index, row in doc.iterrows():
    doc_id = row['docID']
    words = row['reviewText']

    for word in words:
        if word not in inverted_index:
            inverted_index[word] = set()  # Buat set baru untuk kata ini
        inverted_index[word].add(doc_id)  # Tambahkan docID ke set

for word, doc_ids in inverted_index.items():
    print(f"{word}: {', '.join(doc_ids)}")



m: D294, D245, D95, D125, D63, D162, D113, D183, D211, D164, D202, D219, D216, D167, D80, D142, D35, D118, D147, D189, D267, D83, D149, D225, D220, D256, D218, D254, D5, D114, D141, D156, D184, D210, D146, D286, D290, D9, D31, D36, D262, D44, D101, D22, D51, D228, D206, D203, D173, D269, D172, D15, D239, D65, D191, D199, D104, D94, D136, D235, D59, D2, D42, D160, D261, D17, D240, D72, D178, D260, D124, D34, D234, D53, D222, D67, D170, D60, D198, D7, D68, D246, D271, D128, D33, D55, D14, D28, D38, D241, D37, D70, D79, D253, D273, D247, D258, D123, D138, D115, D213, D265, D237, D300, D180, D29, D23, D46, D102, D221, D204, D207, D19, D238, D205, D217, D135, D244, D209, D268, D90, D126, D249, D232, D52, D158, D27, D116, D194, D282, D153, D284, D159, D107, D298, D99, D12, D201, D54, D26, D77, D88, D82, D168, D96, D182, D193, D200, D229, D257, D196, D108, D263, D8, D154, D274, D66, D190, D292, D4, D208, D48, D131, D92, D279, D106, D296, D166, D50, D133, D169, D226, D272, D105, D231, D275, D1

In [130]:

# Kemunculan Kata
documents = doc["reviewText"].astype(str).tolist()
vectorizer = CountVectorizer()
word_counts_matrix = vectorizer.fit_transform(documents)

df_word_counts = pd.DataFrame(word_counts_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_word_counts['docID'] = doc['docID']
df_word_counts = df_word_counts[['docID'] + [col for col in df_word_counts.columns if col != 'docID']]
df_word_counts = df_word_counts.reset_index(drop=True)

df_word_counts.to_csv("/content/drive/MyDrive/dataset/Kemunculan_Kata.csv")

df_word_counts

,docID,abandon,absolut,abut,accept,access,accid,acid,acoust,across,...,yellow,yes,yesterday,yet,young,your,youth,zero,zip,zoom
0,D1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,D2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,D3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,D4,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,D5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,D296,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,D297,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297,D298,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,D299,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
# Pembobotan Kata (TF-IDF)
documents = doc["reviewText"].astype(str).tolist()

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(documents)
df_tfidf = pd.DataFrame(tf_idf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf['docID'] = doc['docID']
df_tfidf = df_tfidf[['docID'] + [col for col in df_tfidf.columns if col != 'docID']]


df_tfidf.to_csv("/content/drive/MyDrive/dataset/Pembobotan_Kata.csv", index=False)
df_tfidf


,docID,abandon,absolut,abut,accept,access,accid,acid,acoust,across,...,yellow,yes,yesterday,yet,young,your,youth,zero,zip,zoom
0,D1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,D2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102176,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,D296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296,D297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,D298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,D299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
# Menentukan/Memasukan Query Yang akan dipakai
sigma = "both english and indonesia man said, sigma sigma on the wall who is the skibidiest of them all"
skibidi = "man, his badass gadget that took a shape of a disk took a long time to buffer"
cenat = "i just feel asleep on china skype meeting yesterday and i have no comment on that"

In [133]:
# memproses query yang akan dimasukan agar sesuai
def query_processing(query):

    query= re.sub('\W',' ',query)
    query= query.strip().lower()
    query= " ".join([word for word in query.split() if word not in stopwords.words('english')])

    return query
print(" Query 1 : " + query_processing(sigma))
print(" Query 2 : " + query_processing(skibidi))
print(" Query 3 : " + query_processing(cenat))

 Query 1 : english indonesia man said sigma sigma wall skibidiest
 Query 2 : man badass gadget took shape disk took long time buffer
 Query 3 : feel asleep china skype meeting yesterday comment


In [134]:
# Menemukan Kesamaan Kosinus Query 1
doc = pd.read_csv("/content/drive/MyDrive/dataset/300DataRev.csv")

documents = doc["reviewText"].astype(str).tolist()
all_texts = documents + [query_processing(sigma)]

vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = vectorizer.fit_transform(all_texts)

query_index = len(all_texts) - 1
query_scores = tf_idf_matrix[query_index]

doc = pd.DataFrame({"docID": [f"d{i+1}" for i in range(len(documents))]})
document_index = doc["docID"].values

def cosine_similarity(vocab_index, document_index, query_scores):
    cosine_scores = {}
    query_scores_array = query_scores.toarray().flatten()
    query_scalar = np.sqrt(np.sum(query_scores_array ** 2))

    for doc_id in document_index:
        doc_index = int(doc_id[1:]) - 1
        doc_vector = vocab_index[doc_index].toarray().flatten()
        doc_scalar = np.sqrt(np.sum(doc_vector ** 2))
        dot_prod = np.sum(doc_vector * query_scores_array)
        cosine = dot_prod / (query_scalar * doc_scalar)
        cosine_scores[doc_id] = cosine

    return pd.Series(cosine_scores)

cosines = cosine_similarity(tf_idf_matrix, document_index, query_scores)
cosines

,0
d1,0.0
d2,0.0
d3,0.0
d4,0.0
d5,0.0
...,...
d296,0.0
d297,0.0
d298,0.0
d299,0.0


In [135]:
# Menemukan Kesamaan Kosinus Query 2
doc = pd.read_csv("/content/drive/MyDrive/dataset/300DataRev.csv")

documents = doc["reviewText"].astype(str).tolist()
all_texts = documents + [query_processing(skibidi)]

vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = vectorizer.fit_transform(all_texts)

query_index = len(all_texts) - 1
query_scores = tf_idf_matrix[query_index]

doc = pd.DataFrame({"docID": [f"d{i+1}" for i in range(len(documents))]})
document_index = doc["docID"].values

def cosine_similarity(vocab_index, document_index, query_scores):
    cosine_scores = {}
    query_scores_array = query_scores.toarray().flatten()
    query_scalar = np.sqrt(np.sum(query_scores_array ** 2))

    for doc_id in document_index:
        doc_index = int(doc_id[1:]) - 1
        doc_vector = vocab_index[doc_index].toarray().flatten()
        doc_scalar = np.sqrt(np.sum(doc_vector ** 2))
        dot_prod = np.sum(doc_vector * query_scores_array)
        cosine = dot_prod / (query_scalar * doc_scalar)
        cosine_scores[doc_id] = cosine

    return pd.Series(cosine_scores)

cosiness = cosine_similarity(tf_idf_matrix, document_index, query_scores)
cosiness

,0
d1,0.000000
d2,0.000000
d3,0.000000
d4,0.000000
d5,0.022941
...,...
d296,0.023863
d297,0.011645
d298,0.000000
d299,0.000000


In [136]:
# Menemukan Kesamaan Kosinus Query 3
doc = pd.read_csv("/content/drive/MyDrive/dataset/300DataRev.csv")

documents = doc["reviewText"].astype(str).tolist()
all_texts = documents + [query_processing(cenat)]

vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = vectorizer.fit_transform(all_texts)

query_index = len(all_texts) - 1
query_scores = tf_idf_matrix[query_index]

doc = pd.DataFrame({"docID": [f"d{i+1}" for i in range(len(documents))]})
document_index = doc["docID"].values

def cosine_similarity(vocab_index, document_index, query_scores):
    cosine_scores = {}
    query_scores_array = query_scores.toarray().flatten()
    query_scalar = np.sqrt(np.sum(query_scores_array ** 2))

    for doc_id in document_index:
        doc_index = int(doc_id[1:]) - 1
        doc_vector = vocab_index[doc_index].toarray().flatten()
        doc_scalar = np.sqrt(np.sum(doc_vector ** 2))
        dot_prod = np.sum(doc_vector * query_scores_array)
        cosine = dot_prod / (query_scalar * doc_scalar)
        cosine_scores[doc_id] = cosine

    return pd.Series(cosine_scores)

cosinesss = cosine_similarity(tf_idf_matrix, document_index, query_scores)
cosinesss

,0
d1,0.000000
d2,0.000000
d3,0.000000
d4,0.042451
d5,0.000000
...,...
d296,0.000000
d297,0.017211
d298,0.082902
d299,0.000000


In [137]:
# Menentukan top 10 dokumen yang paling relevan sesuai dengan ketiga query

def retrieve_index(data,cosine_scores, document_index):

    data = data.copy()
    data['scores'] = cosine_scores.values
    return data.sort_values('scores', ascending=False).head(10).index

indices = retrieve_index(doc, cosines, 'docID')
indicesxd = retrieve_index(doc, cosiness, 'docID')
indiceslmao = retrieve_index(doc, cosinesss, 'docID')

print("query 1: ", indices)
print("query 2: ", indicesxd)
print("query 3: ", indiceslmao)

query 1:  Index([229, 281, 246, 142, 244, 293, 189, 144, 32, 136], dtype='int64')
query 2:  Index([229, 290, 245, 68, 256, 94, 281, 128, 183, 196], dtype='int64')
query 3:  Index([297, 188, 93, 190, 25, 3, 77, 222, 259, 179], dtype='int64')


In [138]:
# Perankingan Query
doc = pd.read_csv("/content/drive/MyDrive/dataset/300DataRev.csv")

def get_query_tfidf(doc, query):
    documents = doc["reviewText"].astype(str).tolist()
    vectorizer = TfidfVectorizer()
    tf_idf_matrix = vectorizer.fit_transform(documents)
    df_tfidf = pd.DataFrame(tf_idf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    query_words = query.split()
    query_tfidf_scores = {}

    for word in query_words:
        if word in df_tfidf.columns:
            query_tfidf_scores[word] = df_tfidf[word].sum()
        else:
            query_tfidf_scores[word] = 0

    query_tfidf_df = pd.DataFrame(query_tfidf_scores.items(), columns=["Word", "TF-IDF"])
    query_tfidf_df = query_tfidf_df.sort_values(by="TF-IDF", ascending=False)
    query_tfidf_df = query_tfidf_df.reset_index(drop=True)
    query_tfidf_df.index = query_tfidf_df.index + 1

    return query_tfidf_df

queryone = get_query_tfidf(doc, query_processing(sigma))
querytwo = get_query_tfidf(doc, query_processing(skibidi))
querythree = get_query_tfidf(doc, query_processing(cenat))


print(queryone)
print(" ")
print(querytwo)
print(" ")
print(querythree)


         Word    TF-IDF
1        said  0.596906
2         man  0.398868
3        wall  0.372161
4     english  0.127076
5   indonesia  0.098647
6       sigma  0.000000
7  skibidiest  0.000000
 
     Word    TF-IDF
1    time  4.031183
2    long  2.708605
3    took  0.665996
4     man  0.398868
5   shape  0.295433
6  buffer  0.245909
7  badass  0.211030
8  gadget  0.100570
9    disk  0.018388
 
        Word    TF-IDF
1       feel  2.271678
2      china  0.289280
3    comment  0.182743
4      skype  0.155244
5  yesterday  0.115397
6     asleep  0.054210
7    meeting  0.000000
